In [1]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, LSTM, GRU, RNN, BatchNormalization, MaxPooling2D, Reshape
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.model_selection import train_test_split

In [2]:
## Loading and visualizing the data

## Loading the dataset
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
print(X_train_valid.shape)
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

## Adjusting the labels so that 

# Cue onset left - 0
# Cue onset right - 1
# Cue onset foot - 2
# Cue onset tongue - 3

y_train_valid -= 769
y_test -= 769

(2115, 22, 1000)


In [3]:
def train_data_prep(X,y,sub_sample,average,noise):
    
    total_X = None
    total_y = None
    
    # Trimming the data (sample,22,1000) -> (sample,22,800)
    X = X[:,:,0:800]
    print('Shape of X after trimming:',X.shape)
    
    # Maxpooling the data (sample,22,800) -> (sample,22,800/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, sub_sample), axis=3)
    
    total_X = X_max
    total_y = y
    print('Shape of X after maxpooling:',total_X.shape)
    
    # Averaging + noise 
    X_average = np.mean(X.reshape(X.shape[0], X.shape[1], -1, average),axis=3)
    X_average = X_average + np.random.normal(0.0, 0.5, X_average.shape)
    
    total_X = np.vstack((total_X, X_average))
    total_y = np.hstack((total_y, y))
    print('Shape of X after averaging + noise and concatenating:',total_X.shape)
    
    # Subsampling
    for i in range(sub_sample):
        X_subsample = X[:, :, i::sub_sample] + \
                            (np.random.normal(0.0, 0.5, X[:, :,i::sub_sample].shape) if noise else 0.0)
            
        total_X = np.vstack((total_X, X_subsample))
        total_y = np.hstack((total_y, y))
    
    print('Shape of X after subsampling and concatenating:', total_X.shape)
    print('Shape of Y:', total_y.shape)
    return total_X, total_y

def test_data_prep(X):
    total_X = None
    
    # Trimming the data (sample,22,1000) -> (sample,22,800)
    X = X[:,:,0:800]
    print('Shape of X after trimming:',X.shape)
    
    # Maxpooling the data (sample,22,800) -> (sample,22,800/sub_sample)
    X_max = np.max(X.reshape(X.shape[0], X.shape[1], -1, 2), axis=3)
    
    total_X = X_max
    print('Shape of X after maxpooling:', total_X.shape)
    return total_X

In [4]:
## Preprocessing the dataset

# Define the model, loss function, and optimizer
x_train, x_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size=0.1)

x_train, y_train = train_data_prep(x_train, y_train, 2, 2, True)
x_valid, y_valid = train_data_prep(x_valid, y_valid, 2, 2, True)
X_test_prep = test_data_prep(X_test)
print('Shape of training set:',x_train.shape)
print('Shape of validation set:',x_valid.shape)
print('Shape of training labels:',y_train.shape)
print('Shape of validation labels:',y_valid.shape)

# Converting the labels to categorical variables for multiclass classification
y_train = to_categorical(y_train, 4)
y_valid = to_categorical(y_valid, 4)
y_test = to_categorical(y_test, 4)
print('Shape of training labels after categorical conversion:',y_train.shape)
print('Shape of validation labels after categorical conversion:',y_valid.shape)
print('Shape of test labels after categorical conversion:',y_test.shape)

# Adding width of the segment to be 1
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
x_valid = x_valid.reshape(x_valid.shape[0], x_valid.shape[1], x_train.shape[2], 1)
x_test = X_test_prep.reshape(X_test_prep.shape[0], X_test_prep.shape[1], X_test_prep.shape[2], 1)
print('Shape of training set after adding width info:',x_train.shape)
print('Shape of validation set after adding width info:',x_valid.shape)
print('Shape of test set after adding width info:',x_test.shape)

# Reshaping the training and validation dataset
x_train = np.swapaxes(x_train, 1,3)
x_train = np.swapaxes(x_train, 1,2)
x_valid = np.swapaxes(x_valid, 1,3)
x_valid = np.swapaxes(x_valid, 1,2)
x_test = np.swapaxes(x_test, 1,3)
x_test = np.swapaxes(x_test, 1,2)
print('Shape of training set after dimension reshaping:',x_train.shape)
print('Shape of validation set after dimension reshaping:',x_valid.shape)
print('Shape of test set after dimension reshaping:',x_test.shape)

Shape of X after trimming: (1903, 22, 800)
Shape of X after maxpooling: (1903, 22, 400)
Shape of X after averaging + noise and concatenating: (3806, 22, 400)
Shape of X after subsampling and concatenating: (7612, 22, 400)
Shape of Y: (7612,)
Shape of X after trimming: (212, 22, 800)
Shape of X after maxpooling: (212, 22, 400)
Shape of X after averaging + noise and concatenating: (424, 22, 400)
Shape of X after subsampling and concatenating: (848, 22, 400)
Shape of Y: (848,)
Shape of X after trimming: (443, 22, 800)
Shape of X after maxpooling: (443, 22, 400)
Shape of training set: (7612, 22, 400)
Shape of validation set: (848, 22, 400)
Shape of training labels: (7612,)
Shape of validation labels: (848,)
Shape of training labels after categorical conversion: (7612, 4)
Shape of validation labels after categorical conversion: (848, 4)
Shape of test labels after categorical conversion: (443, 4)
Shape of training set after adding width info: (7612, 22, 400, 1)
Shape of validation set after 

In [5]:
# Building the CNN model using sequential class
def create_model(cnn_layers, filters, kernel_size, dropout, l2_lambda, gru_layers, gru_size):
    cnn_rnn_model = Sequential()

    cnn_rnn_model.add(Conv2D(filters=filters, kernel_size=kernel_size, padding='same', activation='elu', input_shape=(400,1,22), kernel_regularizer=l2(l2_lambda)))
    cnn_rnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same')) 
    cnn_rnn_model.add(BatchNormalization())
    cnn_rnn_model.add(Dropout(dropout))
    filters *= 2
    
    for i in range(cnn_layers - 1):
        cnn_rnn_model.add(Conv2D(filters=filters, kernel_size=kernel_size, padding='same', activation='elu', kernel_regularizer=l2(l2_lambda)))
        cnn_rnn_model.add(MaxPooling2D(pool_size=(3,1), padding='same'))
        cnn_rnn_model.add(BatchNormalization())
        cnn_rnn_model.add(Dropout(dropout))
        filters *= 2
    
    cnn_rnn_model.add(Flatten()) 
    cnn_rnn_model.add(Dense((gru_size * 2), kernel_regularizer=l2(l2_lambda))) 
    cnn_rnn_model.add(Reshape((gru_size * 2, 1))) 
    #cnn_rnn_model.add(LSTM(10, dropout=0.4, recurrent_dropout=0.1, input_shape=(40,1), return_sequences=False, kernel_regularizer=l2(l2_lambda)))
    
    for i in range(gru_layers - 1):
        cnn_rnn_model.add(GRU(gru_size, dropout=0.4, return_sequences=True, kernel_regularizer=l2(l2_lambda)))
    cnn_rnn_model.add(GRU(gru_size, dropout=0.4, kernel_regularizer=l2(l2_lambda)))
    
    # Output layer with Softmax activation 
    cnn_rnn_model.add(Dense(4, activation='softmax')) # Output FC layer with softmax activation

    return cnn_rnn_model

In [6]:
# Compiling the model
cnn_layers = 3
gru_layers = 2
gru_size= 60
filters = 25
kernel_size = (7,7)
dropout = 0.5
l2_lambda = 0.001

# Model parameters
learning_rate = 1e-3
epochs = 100
cnn_rnn_optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=30, verbose=1, restore_best_weights=True)
# Add early stopping callback to the list of callbacks
callbacks = [early_stopping]

cnn_rnn_model = create_model(cnn_layers, filters, kernel_size, dropout, l2_lambda, gru_layers, gru_size)

cnn_rnn_model.compile(loss='categorical_crossentropy',
                 optimizer=cnn_rnn_optimizer,
                 metrics=['accuracy'])

# Training and validating the model
cnn_rnn_model_results = cnn_rnn_model.fit(x_train,
             y_train,
             batch_size=64,
             epochs=epochs,
             validation_data=(x_valid, y_valid),
             callbacks=callbacks, verbose=True)

## Testing the hybrid CNN-RNN model
cnn_rnn_score = cnn_rnn_model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy of the hybrid CNN-RNN model:', cnn_rnn_score[1])

/Users/danielkong/anaconda3/envs/ECE247/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 21s 155ms/step - accuracy: 0.2806 - loss: 1.5923 - val_accuracy: 0.3632 - val_loss: 1.3981
Epoch 2/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 163ms/step - accuracy: 0.3500 - loss: 1.3737 - val_accuracy: 0.4552 - val_loss: 1.2107
Epoch 3/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 160ms/step - accuracy: 0.3803 - loss: 1.3159 - val_accuracy: 0.4422 - val_loss: 1.2338
Epoch 4/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 160ms/step - accuracy: 0.4041 - loss: 1.2810 - val_accuracy: 0.5318 - val_loss: 1.1294
Epoch 5/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 161ms/step - accuracy: 0.4222 - loss: 1.2561 - val_accuracy: 0.5495 - val_loss: 1.1217
Epoch 6/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 163ms/step - accuracy: 0.4429 - loss: 1.2270 - val_accuracy: 0.5873 - val_loss: 1.0991
Epoch 7/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 160ms/step - accuracy: 0.4670 - loss: 1.2064 - val_accuracy: 0.5448 - val_loss: 1.1318
Epoch 8/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 20s 166ms/step - accuracy: 0.4689 -

In [8]:
# Define early stopping criteria
early_stopping = EarlyStopping(monitor='val_accuracy', patience=20, verbose=1, restore_best_weights=True)
# Add early stopping callback to the list of callbacks
callbacks = [early_stopping]

# Training and validating the model
cnn_rnn_model_results = cnn_rnn_model.fit(x_train,
             y_train,
             batch_size=64,
             epochs=epochs,
             validation_data=(x_valid, y_valid),
             callbacks=callbacks, verbose=True)

## Testing the hybrid CNN-RNN model
cnn_rnn_score = cnn_rnn_model.evaluate(x_test, y_test, verbose=0)
print('Test accuracy of the hybrid CNN-RNN model:', cnn_rnn_score[1])

Epoch 1/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 157ms/step - accuracy: 0.5908 - loss: 1.0143 - val_accuracy: 0.6781 - val_loss: 1.0115
Epoch 2/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 155ms/step - accuracy: 0.5918 - loss: 1.0046 - val_accuracy: 0.6651 - val_loss: 0.9955
Epoch 3/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 18s 153ms/step - accuracy: 0.6091 - loss: 0.9629 - val_accuracy: 0.7134 - val_loss: 0.9672
Epoch 4/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 18s 154ms/step - accuracy: 0.6064 - loss: 0.9767 - val_accuracy: 0.6757 - val_loss: 1.0404
Epoch 5/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 18s 154ms/step - accuracy: 0.5964 - loss: 0.9875 - val_accuracy: 0.6828 - val_loss: 0.9996
Epoch 6/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 18s 155ms/step - accuracy: 0.6091 - loss: 0.9616 - val_accuracy: 0.6887 - val_loss: 1.0086
Epoch 7/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 18s 155ms/step - accuracy: 0.6087 - loss: 0.9625 - val_accuracy: 0.6851 - val_loss: 1.0249
Epoch 8/100
119/119 ━━━━━━━━━━━━━━━━━━━━ 19s 156ms/step - accuracy: 0.6041 -

In [9]:
cnn_rnn_model.save('cnn_gru_model.keras')